[section 1](#section1)<br />
[section 2](#section2)<br />
[section 3](#section3)<br />

In [1]:
!pip install -r requirements.txt

In [2]:
# Imports
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import pandas as pd

# Disable eager execution
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [3]:
# Data processing
from sklearn.model_selection import train_test_split

# Models
import lightgbm as lgb
from sklearn.calibration import CalibratedClassifierCV

# Fairlearn algorithms and utils
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import GridSearch, EqualizedOdds

# Metrics
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

In [4]:
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D #types of layers
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers
from tensorflow.keras import regularizers

<a class="anchor" id="section1"></a>
# Section 1

In [5]:
data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
dataset = pd.read_excel(io=data_url, header=1).drop(columns=['ID']).rename(columns={'PAY_0':'PAY_1'})
dataset.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
# Extract the sensitive feature
A = dataset["SEX"]
A_str = A.map({ 2:"female", 1:"male"})
# Extract the target
Y = dataset["default payment next month"]
categorical_features = ['EDUCATION', 'MARRIAGE','PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
for col in categorical_features:
    dataset[col] = dataset[col].astype('category')

In [7]:
A.describe()

count    30000.000000
mean         1.603733
std          0.489129
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max          2.000000
Name: SEX, dtype: float64

In [8]:
A.head()

0    2
1    2
2    2
3    2
4    1
Name: SEX, dtype: int64

In [9]:
sex_feature = (A - 1).to_numpy()

In [10]:
sex_feature

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [11]:
Y

0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default payment next month, Length: 30000, dtype: int64

In [12]:
dataset

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [13]:
features = dataset.drop(columns=['SEX', 'default payment next month']).copy()

scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

In [14]:
# Train-test split
x_train, x_test, Y_train, Y_test, A_train, A_test, A_str_train, A_str_test, sex_train, sex_split = train_test_split(
    features, 
    Y, 
    A, 
    A_str,
    sex_feature,
    test_size = 0.3, 
    random_state=12345,
    stratify=Y)

In [15]:
x_train[0]

array([-0.75135032,  1.45111372, -1.05729503, -0.16115646,  0.90471219,
        1.78234817,  0.1388648 ,  0.18874609,  0.23491652,  0.25313738,
        0.33679354,  0.33768516,  0.30312013,  0.39388075, -0.16546117,
        0.14258263, -0.34194162, -0.12678393, -0.12641128, -0.21180252,
        0.99493138, -0.15275225])

In [16]:
x_train.shape

(21000, 22)

<a class="anchor" id="section2"></a>
# Section 2

### Build a simple NN model and view sex bias

In [17]:
inputs = keras.Input(shape=(22,))
cls1a = Dense(64, kernel_regularizer=regularizers.L1(1e-5), activation='relu')(inputs)
dropout1 = Dropout(0.1)(cls1a)
cls1b = Dense(32, kernel_regularizer=regularizers.L1(1e-5), activation='relu')(dropout1)
dropout2 = Dropout(0.1)(cls1b)
cls1c = Dense(16, kernel_regularizer=regularizers.L1(1e-5), activation='relu')(dropout2)
output = Dense(1, activation='sigmoid')(cls1c)
model = keras.Model(inputs=inputs, outputs=output, name="simple_model")
model.summary()

Model: "simple_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 22)]              0         
                                                                 
 dense (Dense)               (None, 64)                1472      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                

In [18]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=["accuracy", "mean_absolute_error"]
)

history = model.fit(x_train, Y_train, batch_size=64, epochs=250, validation_split=0.2)

Train on 16800 samples, validate on 4200 samples
Epoch 1/250
16800/16800 [==============================] - ETA: 0s - loss: 0.5080 - accuracy: 0.7908 - mean_absolute_error: 0.3287

C:\Users\netwo\anaconda3\envs\Thesis-minimum\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


16800/16800 [==============================] - 3s 197us/sample - loss: 0.5080 - accuracy: 0.7908 - mean_absolute_error: 0.3287 - val_loss: 0.4637 - val_accuracy: 0.8100 - val_mean_absolute_error: 0.2899
Epoch 2/250
16800/16800 [==============================] - 2s 129us/sample - loss: 0.4627 - accuracy: 0.8132 - mean_absolute_error: 0.2897 - val_loss: 0.4537 - val_accuracy: 0.8126 - val_mean_absolute_error: 0.2804
Epoch 3/250
16800/16800 [==============================] - 2s 137us/sample - loss: 0.4526 - accuracy: 0.8146 - mean_absolute_error: 0.2830 - val_loss: 0.4460 - val_accuracy: 0.8183 - val_mean_absolute_error: 0.2715
Epoch 4/250
16800/16800 [==============================] - 2s 118us/sample - loss: 0.4469 - accuracy: 0.8153 - mean_absolute_error: 0.2781 - val_loss: 0.4436 - val_accuracy: 0.8169 - val_mean_absolute_error: 0.2767
Epoch 5/250
16800/16800 [==============================] - 2s 122us/sample - loss: 0.4435 - accuracy: 0.8178 - mean_absolute_error: 0.2765 - val_loss: 0

16800/16800 [==============================] - 2s 130us/sample - loss: 0.4167 - accuracy: 0.8251 - mean_absolute_error: 0.2582 - val_loss: 0.4472 - val_accuracy: 0.8152 - val_mean_absolute_error: 0.2779
Epoch 40/250
16800/16800 [==============================] - 2s 137us/sample - loss: 0.4189 - accuracy: 0.8251 - mean_absolute_error: 0.2608 - val_loss: 0.4462 - val_accuracy: 0.8124 - val_mean_absolute_error: 0.2748
Epoch 41/250
16800/16800 [==============================] - 2s 133us/sample - loss: 0.4182 - accuracy: 0.8264 - mean_absolute_error: 0.2591 - val_loss: 0.4480 - val_accuracy: 0.8126 - val_mean_absolute_error: 0.2770
Epoch 42/250
16800/16800 [==============================] - 2s 128us/sample - loss: 0.4163 - accuracy: 0.8270 - mean_absolute_error: 0.2581 - val_loss: 0.4480 - val_accuracy: 0.8107 - val_mean_absolute_error: 0.2761
Epoch 43/250
16800/16800 [==============================] - 2s 132us/sample - loss: 0.4158 - accuracy: 0.8258 - mean_absolute_error: 0.2583 - val_los

Epoch 77/250
16800/16800 [==============================] - 2s 128us/sample - loss: 0.4057 - accuracy: 0.8302 - mean_absolute_error: 0.2522 - val_loss: 0.4570 - val_accuracy: 0.8126 - val_mean_absolute_error: 0.2602
Epoch 78/250
16800/16800 [==============================] - 2s 129us/sample - loss: 0.4049 - accuracy: 0.8307 - mean_absolute_error: 0.2499 - val_loss: 0.4543 - val_accuracy: 0.8124 - val_mean_absolute_error: 0.2653
Epoch 79/250
16800/16800 [==============================] - 2s 124us/sample - loss: 0.4062 - accuracy: 0.8286 - mean_absolute_error: 0.2516 - val_loss: 0.4576 - val_accuracy: 0.8129 - val_mean_absolute_error: 0.2664
Epoch 80/250
16800/16800 [==============================] - 2s 121us/sample - loss: 0.4073 - accuracy: 0.8290 - mean_absolute_error: 0.2526 - val_loss: 0.4568 - val_accuracy: 0.8145 - val_mean_absolute_error: 0.2662
Epoch 81/250
16800/16800 [==============================] - 2s 124us/sample - loss: 0.4031 - accuracy: 0.8316 - mean_absolute_error: 0.2

Epoch 115/250
16800/16800 [==============================] - 2s 122us/sample - loss: 0.3961 - accuracy: 0.8330 - mean_absolute_error: 0.2446 - val_loss: 0.4660 - val_accuracy: 0.8114 - val_mean_absolute_error: 0.2633
Epoch 116/250
16800/16800 [==============================] - 2s 130us/sample - loss: 0.3960 - accuracy: 0.8323 - mean_absolute_error: 0.2436 - val_loss: 0.4667 - val_accuracy: 0.8048 - val_mean_absolute_error: 0.2783
Epoch 117/250
16800/16800 [==============================] - 2s 124us/sample - loss: 0.3967 - accuracy: 0.8326 - mean_absolute_error: 0.2456 - val_loss: 0.4651 - val_accuracy: 0.8098 - val_mean_absolute_error: 0.2643
Epoch 118/250
16800/16800 [==============================] - 2s 121us/sample - loss: 0.3926 - accuracy: 0.8338 - mean_absolute_error: 0.2425 - val_loss: 0.4625 - val_accuracy: 0.8121 - val_mean_absolute_error: 0.2654
Epoch 119/250
16800/16800 [==============================] - 2s 122us/sample - loss: 0.3963 - accuracy: 0.8325 - mean_absolute_error

Epoch 153/250
16800/16800 [==============================] - 2s 118us/sample - loss: 0.3889 - accuracy: 0.8347 - mean_absolute_error: 0.2389 - val_loss: 0.4652 - val_accuracy: 0.8071 - val_mean_absolute_error: 0.2688
Epoch 154/250
16800/16800 [==============================] - 2s 125us/sample - loss: 0.3879 - accuracy: 0.8355 - mean_absolute_error: 0.2391 - val_loss: 0.4700 - val_accuracy: 0.8076 - val_mean_absolute_error: 0.2614
Epoch 155/250
16800/16800 [==============================] - 2s 119us/sample - loss: 0.3882 - accuracy: 0.8354 - mean_absolute_error: 0.2394 - val_loss: 0.4714 - val_accuracy: 0.8069 - val_mean_absolute_error: 0.2616
Epoch 156/250
16800/16800 [==============================] - 2s 115us/sample - loss: 0.3895 - accuracy: 0.8337 - mean_absolute_error: 0.2402 - val_loss: 0.4683 - val_accuracy: 0.8074 - val_mean_absolute_error: 0.2670
Epoch 157/250
16800/16800 [==============================] - 2s 114us/sample - loss: 0.3869 - accuracy: 0.8350 - mean_absolute_error

Epoch 191/250
16800/16800 [==============================] - 2s 123us/sample - loss: 0.3827 - accuracy: 0.8415 - mean_absolute_error: 0.2356 - val_loss: 0.4775 - val_accuracy: 0.8036 - val_mean_absolute_error: 0.2724
Epoch 192/250
16800/16800 [==============================] - 2s 117us/sample - loss: 0.3818 - accuracy: 0.8371 - mean_absolute_error: 0.2348 - val_loss: 0.4807 - val_accuracy: 0.8067 - val_mean_absolute_error: 0.2675
Epoch 193/250
16800/16800 [==============================] - 2s 122us/sample - loss: 0.3824 - accuracy: 0.8381 - mean_absolute_error: 0.2356 - val_loss: 0.4864 - val_accuracy: 0.8055 - val_mean_absolute_error: 0.2569
Epoch 194/250
16800/16800 [==============================] - 2s 119us/sample - loss: 0.3838 - accuracy: 0.8376 - mean_absolute_error: 0.2357 - val_loss: 0.4812 - val_accuracy: 0.8040 - val_mean_absolute_error: 0.2652
Epoch 195/250
16800/16800 [==============================] - 2s 117us/sample - loss: 0.3844 - accuracy: 0.8380 - mean_absolute_error

Epoch 229/250
16800/16800 [==============================] - 2s 114us/sample - loss: 0.3820 - accuracy: 0.8407 - mean_absolute_error: 0.2346 - val_loss: 0.4838 - val_accuracy: 0.8052 - val_mean_absolute_error: 0.2642
Epoch 230/250
16800/16800 [==============================] - 2s 115us/sample - loss: 0.3794 - accuracy: 0.8403 - mean_absolute_error: 0.2330 - val_loss: 0.4883 - val_accuracy: 0.8024 - val_mean_absolute_error: 0.2668
Epoch 231/250
16800/16800 [==============================] - 2s 114us/sample - loss: 0.3767 - accuracy: 0.8414 - mean_absolute_error: 0.2320 - val_loss: 0.4970 - val_accuracy: 0.8067 - val_mean_absolute_error: 0.2578
Epoch 232/250
16800/16800 [==============================] - 2s 114us/sample - loss: 0.3798 - accuracy: 0.8413 - mean_absolute_error: 0.2330 - val_loss: 0.4901 - val_accuracy: 0.8043 - val_mean_absolute_error: 0.2634
Epoch 233/250
16800/16800 [==============================] - 2s 116us/sample - loss: 0.3769 - accuracy: 0.8417 - mean_absolute_error

### Get statistics on simple nn

In [19]:
# Scores on test set
test_scores = model.predict(x_test)

C:\Users\netwo\anaconda3\envs\Thesis-minimum\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [20]:
test_scores

array([[0.02858746],
       [0.3252348 ],
       [0.37272477],
       ...,
       [0.16782422],
       [0.07963651],
       [0.06422914]], dtype=float32)

In [21]:
# Train AUC
roc_auc_score(Y_train, model.predict(x_train))

0.8599222577374512

In [22]:
# Predictions (0 or 1) on test set
test_preds = (test_scores >= np.mean(Y_train)) * 1

In [23]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    Y_test, test_preds, sensitive_features=A_str_test)

mf.by_group

C:\Users\netwo\anaconda3\envs\Thesis-minimum\lib\site-packages\fairlearn\metrics\_metric_frame.py:63: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  warnings.warn(f"You have provided {args_msg} as positional arguments. "


,FPR,FNR
SEX,,
female,0.20902,0.395105
male,0.241954,0.381346


In [24]:
# Helper functions
def get_metrics_df(models_dict, y_true, group):
    metrics_dict = {
        "Overall selection rate": (
            lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        "Overall balanced error rate": (
            lambda x: 1-balanced_accuracy_score(y_true, x), True),
        "Balanced error rate difference": (
            lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        " ------": (lambda x: "", True),
        "False positive rate difference": (
            lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
        "False negative rate difference": (
            lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "AUC difference": (
            lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
        df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores) 
                                for model_name, (preds, scores) in models_dict.items()]
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

In [25]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, Y_test, A_str_test)

,Unmitigated
Overall selection rate,0.307556
Demographic parity difference,0.042246
Demographic parity ratio,0.873298
------,
Overall balanced error rate,0.305341
Balanced error rate difference,0.009587
------,
False positive rate difference,0.032934
False negative rate difference,0.013759
Equalized odds difference,0.032934


<a class="anchor" id="section3"></a>

# SECTION 3: Use a split-model

In [26]:
inputs = keras.Input(shape=(22,), name='inputs')

cls1a = Dense(64, name='cls1a', kernel_regularizer=regularizers.L1(1e-5), activation='relu')(inputs)
dropout1a = Dropout(0.1, name='dropout1a')(cls1a)
cls1b = Dense(32, name='cls1b', kernel_regularizer=regularizers.L1(1e-5), activation='relu')(dropout1a)
dropout1b = Dropout(0.1, name='dropout1b')(cls1b)
cls1c = Dense(16, name='cls1c', kernel_regularizer=regularizers.L1(1e-5), activation='relu')(dropout1b)
output1 = Dense(1, name='output1', activation='sigmoid')(cls1c)

cls2a = Dense(64, name='cls2a', kernel_regularizer=regularizers.L1(1e-5), activation='relu')(inputs)
dropout2a = Dropout(0.1, name='dropout2a')(cls2a)
cls2b = Dense(32, name='cls2b', kernel_regularizer=regularizers.L1(1e-5), activation='relu')(dropout2a)
dropout2b = Dropout(0.1, name='dropout2b')(cls2b)
cls2c = Dense(16, name='cls2c', kernel_regularizer=regularizers.L1(1e-5), activation='relu')(dropout2b)
output2 = Dense(1, name='output2', activation='sigmoid')(cls2c)

model = keras.Model(inputs=inputs, outputs=[output1, output2], name="simple_model")
model.summary()

Model: "simple_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 22)]         0           []                               
                                                                                                  
 cls1a (Dense)                  (None, 64)           1472        ['inputs[0][0]']                 
                                                                                                  
 cls2a (Dense)                  (None, 64)           1472        ['inputs[0][0]']                 
                                                                                                  
 dropout1a (Dropout)            (None, 64)           0           ['cls1a[0][0]']                  
                                                                                       

In [27]:
def orthogonal_loss(y_true, y_pred):
    # requires cls1, cls2 layers to be defined through functional API
    _cls1 = cls1a
    _cls2 = cls2a
    cls1_cls2 = tf.linalg.matmul(_cls1, _cls2, transpose_b=True)
    numerator = tf.norm(cls1_cls2, ord=1)
    cls1_norm = tf.norm(_cls1, ord=2)
    cls2_norm = tf.norm(_cls2, ord=2)
    denominator = tf.math.multiply(cls1_norm, cls2_norm)
    loss_ortho = tf.math.divide_no_nan(numerator, denominator)
    
    loss_categorical = keras.losses.binary_crossentropy(y_true, y_pred)
    
    return loss_categorical + loss_ortho

In [28]:
model.compile(
    loss=orthogonal_loss,
    optimizer='adam',
    metrics=["accuracy", "mean_absolute_error"]
)

history = model.fit(x_train, [Y_train, sex_train] , batch_size=64, epochs=250, validation_split=0.2)

Train on 16800 samples, validate on 4200 samples
Epoch 1/250
16768/16800 [============================>.] - ETA: 0s - loss: 10.3647 - output1_loss: 5.1059 - output2_loss: 5.2486 - output1_accuracy: 0.7773 - output1_mean_absolute_error: 0.3620 - output2_accuracy: 0.5943 - output2_mean_absolute_error: 0.4818

C:\Users\netwo\anaconda3\envs\Thesis-minimum\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


16800/16800 [==============================] - 5s 273us/sample - loss: 10.3480 - output1_loss: 5.0891 - output2_loss: 5.2320 - output1_accuracy: 0.7774 - output1_mean_absolute_error: 0.3620 - output2_accuracy: 0.5943 - output2_mean_absolute_error: 0.4818 - val_loss: 2.1961 - val_output1_loss: 0.9961 - val_output2_loss: 1.1865 - val_output1_accuracy: 0.8000 - val_output1_mean_absolute_error: 0.3238 - val_output2_accuracy: 0.5952 - val_output2_mean_absolute_error: 0.4794
Epoch 2/250
16800/16800 [==============================] - 4s 244us/sample - loss: 1.5210 - output1_loss: 0.6604 - output2_loss: 0.8503 - output1_accuracy: 0.7987 - output1_mean_absolute_error: 0.3116 - output2_accuracy: 0.5999 - output2_mean_absolute_error: 0.4772 - val_loss: 1.2559 - val_output1_loss: 0.5203 - val_output2_loss: 0.7267 - val_output1_accuracy: 0.8033 - val_output1_mean_absolute_error: 0.3024 - val_output2_accuracy: 0.5948 - val_output2_mean_absolute_error: 0.4789
Epoch 3/250
16800/16800 [================

Epoch 18/250
16800/16800 [==============================] - 4s 241us/sample - loss: 1.0889 - output1_loss: 0.4326 - output2_loss: 0.6494 - output1_accuracy: 0.8188 - output1_mean_absolute_error: 0.2722 - output2_accuracy: 0.6232 - output2_mean_absolute_error: 0.4583 - val_loss: 1.0971 - val_output1_loss: 0.4389 - val_output2_loss: 0.6503 - val_output1_accuracy: 0.8160 - val_output1_mean_absolute_error: 0.2704 - val_output2_accuracy: 0.6202 - val_output2_mean_absolute_error: 0.4572
Epoch 19/250
16800/16800 [==============================] - 4s 250us/sample - loss: 1.0883 - output1_loss: 0.4329 - output2_loss: 0.6485 - output1_accuracy: 0.8173 - output1_mean_absolute_error: 0.2717 - output2_accuracy: 0.6256 - output2_mean_absolute_error: 0.4571 - val_loss: 1.0951 - val_output1_loss: 0.4393 - val_output2_loss: 0.6493 - val_output1_accuracy: 0.8167 - val_output1_mean_absolute_error: 0.2755 - val_output2_accuracy: 0.6238 - val_output2_mean_absolute_error: 0.4568
Epoch 20/250
16800/16800 [==

Epoch 35/250
16800/16800 [==============================] - 4s 246us/sample - loss: 1.0657 - output1_loss: 0.4230 - output2_loss: 0.6359 - output1_accuracy: 0.8207 - output1_mean_absolute_error: 0.2664 - output2_accuracy: 0.6364 - output2_mean_absolute_error: 0.4460 - val_loss: 1.0908 - val_output1_loss: 0.4400 - val_output2_loss: 0.6445 - val_output1_accuracy: 0.8107 - val_output1_mean_absolute_error: 0.2801 - val_output2_accuracy: 0.6300 - val_output2_mean_absolute_error: 0.4504
Epoch 36/250
16800/16800 [==============================] - 4s 242us/sample - loss: 1.0656 - output1_loss: 0.4226 - output2_loss: 0.6360 - output1_accuracy: 0.8206 - output1_mean_absolute_error: 0.2661 - output2_accuracy: 0.6407 - output2_mean_absolute_error: 0.4458 - val_loss: 1.0932 - val_output1_loss: 0.4389 - val_output2_loss: 0.6472 - val_output1_accuracy: 0.8133 - val_output1_mean_absolute_error: 0.2688 - val_output2_accuracy: 0.6248 - val_output2_mean_absolute_error: 0.4439
Epoch 37/250
16800/16800 [==

...

Epoch 171/250
16800/16800 [==============================] - 4s 242us/sample - loss: 1.0183 - output1_loss: 0.3992 - output2_loss: 0.6095 - output1_accuracy: 0.8302 - output1_mean_absolute_error: 0.2521 - output2_accuracy: 0.6644 - output2_mean_absolute_error: 0.4216 - val_loss: 1.1128 - val_output1_loss: 0.4512 - val_output2_loss: 0.6527 - val_output1_accuracy: 0.8098 - val_output1_mean_absolute_error: 0.2598 - val_output2_accuracy: 0.6217 - val_output2_mean_absolute_error: 0.4448
Epoch 172/250
16800/16800 [==============================] - 4s 244us/sample - loss: 1.0163 - output1_loss: 0.3979 - output2_loss: 0.6091 - output1_accuracy: 0.8288 - output1_mean_absolute_error: 0.2500 - output2_accuracy: 0.6662 - output2_mean_absolute_error: 0.4226 - val_loss: 1.1159 - val_output1_loss: 0.4512 - val_output2_loss: 0.6547 - val_output1_accuracy: 0.8095 - val_output1_mean_absolute_error: 0.2673 - val_output2_accuracy: 0.6243 - val_output2_mean_absolute_error: 0.4431
Epoch 173/250
16800/16800 

Epoch 188/250
16800/16800 [==============================] - 4s 239us/sample - loss: 1.0152 - output1_loss: 0.3971 - output2_loss: 0.6084 - output1_accuracy: 0.8298 - output1_mean_absolute_error: 0.2498 - output2_accuracy: 0.6666 - output2_mean_absolute_error: 0.4225 - val_loss: 1.1170 - val_output1_loss: 0.4533 - val_output2_loss: 0.6533 - val_output1_accuracy: 0.8088 - val_output1_mean_absolute_error: 0.2653 - val_output2_accuracy: 0.6240 - val_output2_mean_absolute_error: 0.4449
Epoch 189/250
16800/16800 [==============================] - 4s 250us/sample - loss: 1.0141 - output1_loss: 0.3962 - output2_loss: 0.6079 - output1_accuracy: 0.8290 - output1_mean_absolute_error: 0.2497 - output2_accuracy: 0.6676 - output2_mean_absolute_error: 0.4214 - val_loss: 1.1211 - val_output1_loss: 0.4583 - val_output2_loss: 0.6528 - val_output1_accuracy: 0.8090 - val_output1_mean_absolute_error: 0.2625 - val_output2_accuracy: 0.6198 - val_output2_mean_absolute_error: 0.4456
Epoch 190/250
16800/16800 

Epoch 205/250
16800/16800 [==============================] - 4s 253us/sample - loss: 1.0140 - output1_loss: 0.3965 - output2_loss: 0.6072 - output1_accuracy: 0.8302 - output1_mean_absolute_error: 0.2489 - output2_accuracy: 0.6716 - output2_mean_absolute_error: 0.4199 - val_loss: 1.1250 - val_output1_loss: 0.4594 - val_output2_loss: 0.6547 - val_output1_accuracy: 0.8095 - val_output1_mean_absolute_error: 0.2601 - val_output2_accuracy: 0.6224 - val_output2_mean_absolute_error: 0.4419
Epoch 206/250
16800/16800 [==============================] - 4s 241us/sample - loss: 1.0084 - output1_loss: 0.3949 - output2_loss: 0.6036 - output1_accuracy: 0.8308 - output1_mean_absolute_error: 0.2479 - output2_accuracy: 0.6710 - output2_mean_absolute_error: 0.4182 - val_loss: 1.1282 - val_output1_loss: 0.4578 - val_output2_loss: 0.6588 - val_output1_accuracy: 0.8090 - val_output1_mean_absolute_error: 0.2619 - val_output2_accuracy: 0.6221 - val_output2_mean_absolute_error: 0.4360
Epoch 207/250
16800/16800 

Epoch 222/250
16800/16800 [==============================] - 4s 243us/sample - loss: 1.0105 - output1_loss: 0.3948 - output2_loss: 0.6054 - output1_accuracy: 0.8311 - output1_mean_absolute_error: 0.2487 - output2_accuracy: 0.6704 - output2_mean_absolute_error: 0.4187 - val_loss: 1.1259 - val_output1_loss: 0.4581 - val_output2_loss: 0.6578 - val_output1_accuracy: 0.8043 - val_output1_mean_absolute_error: 0.2661 - val_output2_accuracy: 0.6176 - val_output2_mean_absolute_error: 0.4408
Epoch 223/250
16800/16800 [==============================] - 4s 243us/sample - loss: 1.0093 - output1_loss: 0.3961 - output2_loss: 0.6025 - output1_accuracy: 0.8304 - output1_mean_absolute_error: 0.2488 - output2_accuracy: 0.6732 - output2_mean_absolute_error: 0.4181 - val_loss: 1.1246 - val_output1_loss: 0.4558 - val_output2_loss: 0.6581 - val_output1_accuracy: 0.8050 - val_output1_mean_absolute_error: 0.2738 - val_output2_accuracy: 0.6193 - val_output2_mean_absolute_error: 0.4419
Epoch 224/250
16800/16800 

Epoch 239/250
16800/16800 [==============================] - 4s 242us/sample - loss: 1.0073 - output1_loss: 0.3932 - output2_loss: 0.6039 - output1_accuracy: 0.8302 - output1_mean_absolute_error: 0.2478 - output2_accuracy: 0.6724 - output2_mean_absolute_error: 0.4182 - val_loss: 1.1244 - val_output1_loss: 0.4577 - val_output2_loss: 0.6550 - val_output1_accuracy: 0.8060 - val_output1_mean_absolute_error: 0.2746 - val_output2_accuracy: 0.6188 - val_output2_mean_absolute_error: 0.4407
Epoch 240/250
16800/16800 [==============================] - 4s 248us/sample - loss: 1.0039 - output1_loss: 0.3911 - output2_loss: 0.6017 - output1_accuracy: 0.8321 - output1_mean_absolute_error: 0.2471 - output2_accuracy: 0.6704 - output2_mean_absolute_error: 0.4170 - val_loss: 1.1350 - val_output1_loss: 0.4622 - val_output2_loss: 0.6620 - val_output1_accuracy: 0.8083 - val_output1_mean_absolute_error: 0.2622 - val_output2_accuracy: 0.6214 - val_output2_mean_absolute_error: 0.4385
Epoch 241/250
16800/16800 

# Get metrics on new predictions

In [29]:
# Scores on test set
test_scores = model.predict(x_test)[0]

C:\Users\netwo\anaconda3\envs\Thesis-minimum\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [30]:
test_scores

array([[0.0657366 ],
       [0.22863008],
       [0.14244564],
       ...,
       [0.50963557],
       [0.02086125],
       [0.05090822]], dtype=float32)

In [31]:
# Train AUC
roc_auc_score(Y_train, model.predict(x_train)[0])

0.8348078475456594

In [32]:
# Predictions (0 or 1) on test set
test_preds = (test_scores >= np.mean(Y_test)) * 1

In [33]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    Y_test, test_preds, sensitive_features=A_str_test)

mf.by_group

C:\Users\netwo\anaconda3\envs\Thesis-minimum\lib\site-packages\fairlearn\metrics\_metric_frame.py:63: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  warnings.warn(f"You have provided {args_msg} as positional arguments. "


,FPR,FNR
SEX,,
female,0.191163,0.399476
male,0.228323,0.400236


In [34]:
# Helper functions
def get_metrics_df(models_dict, y_true, group):
    metrics_dict = {
        "Overall selection rate": (
            lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        "Overall balanced error rate": (
            lambda x: 1-balanced_accuracy_score(y_true, x), True),
        "Balanced error rate difference": (
            lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        " ------": (lambda x: "", True),
        "False positive rate difference": (
            lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
        "False negative rate difference": (
            lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "AUC difference": (
            lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
        df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores) 
                                for model_name, (preds, scores) in models_dict.items()]
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

In [35]:
# Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, Y_test, A_str_test)

,Unmitigated
Overall selection rate,0.292556
Demographic parity difference,0.042396
Demographic parity ratio,0.866898
------,
Overall balanced error rate,0.302482
Balanced error rate difference,0.01896
------,
False positive rate difference,0.03716
False negative rate difference,0.000761
Equalized odds difference,0.03716
